In [1]:
import numpy
import numpy as np
import pandas
import pandas as pd
from sklearn import preprocessing
import atmos

In [2]:
train_X = pandas.read_csv('./train_sensor_data_v1.csv',encoding = 'unicode_escape')
train_X.drop('ID',inplace=True, axis=1)

train_y = pandas.read_csv('./train_actuator_data_v1.csv',encoding = 'unicode_escape',dtype=str)
train_y.drop('ID',inplace=True, axis=1)

val_X = pandas.read_csv('./val_sensor_data_v1.csv',encoding = 'unicode_escape')
val_X.drop('ID',inplace=True, axis=1)

val_y = pandas.read_csv('./val_actuator_data_v1.csv',encoding = 'unicode_escape',dtype=str)
val_y.drop('ID',inplace=True,axis=1)

test_X = pandas.read_csv('./test_sensor_data_v1.csv',encoding = 'unicode_escape')
test_X.drop('ID',inplace=True,axis=1)

train_X

,A_CO2,A_TEMP,A_HUMI,A_LIGHT1,A_LIGHT2,B_CO2,B_TEMP,B_HUMI,B_LIGHT1,B_LIGHT2,CO2,C_CO2,C_TEMP,C_HUMI,C_LIGHT1,C_LIGHT2,TEMP,HUMI
0,5000.0,18.0,75.0,7,0,5000,18.0,80.0,11,0,6338.000000,5000.0,18.0,75.0,17,0,17.900000,76.099998
1,3000.0,13.0,78.0,7,0,3000,13.5,80.0,11,0,3000.000000,3000.0,13.0,80.0,17,0,12.000000,89.199997
2,3000.0,13.0,78.0,7,0,3000,13.5,80.0,11,0,2138.000000,3000.0,13.0,80.0,17,0,12.500000,93.099998
3,6000.0,18.0,90.0,7,0,6000,18.0,90.0,13,0,64882.000000,6000.0,18.0,90.0,18,0,18.000000,84.800003
4,6000.0,14.0,90.0,7,0,6000,14.5,86.0,11,0,6094.000000,6000.0,14.0,86.0,17,0,14.900000,96.300003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234451,6000.0,18.0,80.0,6,0,6000,18.0,80.0,11,0,993.366638,6000.0,18.0,80.0,22,0,17.951666,93.055000
234452,3000.0,13.5,80.0,7,0,3000,14.5,80.0,11,0,5232.000000,3000.0,14.0,80.0,17,0,14.400000,86.199997
234453,3000.0,13.0,75.0,8,30,3000,13.0,75.0,13,0,690.000000,3000.0,13.0,75.0,22,0,23.500000,101.900002
234454,6000.0,18.0,90.0,7,0,6000,18.0,90.0,13,0,64882.000000,6000.0,18.0,90.0,18,0,17.299999,86.000000


In [3]:
train_X['CO2_outliers'] = train_X['CO2'].apply(lambda x: 1 if x > 60000 else 0)
val_X['CO2_outliers'] = val_X['CO2'].apply(lambda x: 1 if x > 60000 else 0)
test_X['CO2_outliers'] = test_X['CO2'].apply(lambda x: 1 if x > 60000 else 0)

train_X['CO2'] = train_X['CO2'].apply(lambda x: x/10 if x > 60000 else x)
val_X['CO2'] = val_X['CO2'].apply(lambda x: x/10 if x > 60000 else x)
test_X['CO2'] = test_X['CO2'].apply(lambda x: x/10 if x > 60000 else x)

sensors = ['CO2','TEMP','HUMI']
times = ['A','B','C']
for f in sensors:
    tABC,vABC,fABC = 0,0,0
    for i,t in enumerate(times):
        # train_X['diffA_CO2'] = (train_X['CO2']-train_X['A_CO2']) / (train_X['A_CO2'])
        train_X['diff'+t+'_'+f] = ( train_X[f] - train_X[t+'_'+f] ) / ( train_X[t+'_'+f] )
        val_X['diff'+t+'_'+f] = ( val_X[f] - val_X[t+'_'+f] ) / ( val_X[t+'_'+f] )
        test_X['diff'+t+'_'+f] = ( test_X[f] - test_X[t+'_'+f] ) / ( test_X[t+'_'+f] )
        train_X['rdiff'+t+'_'+f] = ( train_X[t+'_'+f] - train_X[f] ) / ( train_X[f] )
        val_X['rdiff'+t+'_'+f] = ( val_X[t+'_'+f] - val_X[f] ) / ( val_X[f] )
        test_X['rdiff'+t+'_'+f] = ( test_X[t+'_'+f] - test_X[f] ) / ( test_X[f] )
        
        
        #A+B or B+C or A+C
        next_time_str = times[(i+1)%3]
        train_X['diff'+t+next_time_str+'_'+f] = ( train_X[f] - ( train_X[t+'_'+f]+ train_X[next_time_str+'_'+f] )/2 )  / ( (train_X[t+'_'+f]+ train_X[next_time_str+'_'+f])/2 )
        val_X['diff'+t+next_time_str+'_'+f] = ( val_X[f] - ( val_X[t+'_'+f]+ val_X[next_time_str+'_'+f] )/2 )  / ( (val_X[t+'_'+f]+ val_X[next_time_str+'_'+f])/2 )
        test_X['diff'+t+next_time_str+'_'+f] = ( test_X[f] - ( test_X[t+'_'+f]+ test_X[next_time_str+'_'+f] )/2 )  / ( (test_X[t+'_'+f]+ test_X[next_time_str+'_'+f])/2 )
        train_X['diffv'+t+next_time_str+'_'+f] = ( train_X[f] - ( train_X[t+'_'+f]+ train_X[next_time_str+'_'+f] )/2 ) 
        val_X['diffv'+t+next_time_str+'_'+f] = ( val_X[f] - ( val_X[t+'_'+f]+ val_X[next_time_str+'_'+f] )/2 )  
        test_X['diffv'+t+next_time_str+'_'+f] = ( test_X[f] - ( test_X[t+'_'+f]+ test_X[next_time_str+'_'+f] )/2 )  
        
        train_X['agg'+t+next_time_str+'_'+f] = ( train_X[t+'_'+f]+ train_X[next_time_str+'_'+f] )
        val_X['agg'+t+next_time_str+'_'+f] = ( val_X[t+'_'+f]+ val_X[next_time_str+'_'+f] ) 
        test_X['agg'+t+next_time_str+'_'+f] =  ( test_X[t+'_'+f]+ test_X[next_time_str+'_'+f] )
        train_X['agg'+t+next_time_str+'_'+f] = ( train_X[t+'_'+f]+ train_X[next_time_str+'_'+f] )
        val_X['agg'+t+next_time_str+'_'+f] = (  val_X[t+'_'+f]+ val_X[next_time_str+'_'+f] ) 
        test_X['agg'+t+next_time_str+'_'+f] = (  test_X[t+'_'+f]+ test_X[next_time_str+'_'+f] )
        
        
        train_X['rdiff'+t+next_time_str+'_'+f] = ( ( train_X[t+'_'+f]+ train_X[next_time_str+'_'+f] )/2 - train_X[f] )  / ( train_X[f] )
        val_X['rdiff'+t+next_time_str+'_'+f] = ( ( val_X[t+'_'+f]+ val_X[next_time_str+'_'+f] )/2 - val_X[f] )  / ( val_X[f] )
        test_X['rdiff'+t+next_time_str+'_'+f] = ( ( test_X[t+'_'+f]+ test_X[next_time_str+'_'+f] )/2 - test_X[f] )  / ( test_X[f] )
        train_X['rdiffv'+t+next_time_str+'_'+f] = ( ( train_X[t+'_'+f]+ train_X[next_time_str+'_'+f] )/2 - train_X[f] ) 
        val_X['rdiffv'+t+next_time_str+'_'+f] = ( ( val_X[t+'_'+f]+ val_X[next_time_str+'_'+f] )/2 - val_X[f] )  
        test_X['rdiffv'+t+next_time_str+'_'+f] = ( ( test_X[t+'_'+f]+ test_X[next_time_str+'_'+f] )/2 - test_X[f] )
        # A+B+C
        tABC += train_X[t+'_'+f]
        vABC += val_X[t+'_'+f] 
        fABC += test_X[t+'_'+f]
        
    train_X['diff_'+f] = ( train_X[f] - tABC/3 ) / ( tABC/3 )
    train_X['aggABC_'+f] = tABC
    val_X['diff_'+f] = ( val_X[f] - vABC/3 ) / ( vABC/3 )
    val_X['aggABC_'+f] = vABC
    test_X['diff_'+f] = ( test_X[f] - fABC/3 ) / ( fABC/3 )
    test_X['aggABC_'+f] = fABC
    train_X['rdiff_'+f] = (tABC/3 - train_X[f] ) / ( tABC )
    val_X['rdiff_'+f] = ( vABC/3 - val_X[f] ) / ( vABC )
    test_X['rdiff_'+f] = ( fABC/3 - test_X[f] ) / ( fABC )
    
train_X

,A_CO2,A_TEMP,A_HUMI,A_LIGHT1,A_LIGHT2,B_CO2,B_TEMP,B_HUMI,B_LIGHT1,B_LIGHT2,...,diffC_HUMI,rdiffC_HUMI,diffCA_HUMI,diffvCA_HUMI,aggCA_HUMI,rdiffCA_HUMI,rdiffvCA_HUMI,diff_HUMI,aggABC_HUMI,rdiff_HUMI
0,5000.0,18.0,75.0,7,0,5000,18.0,80.0,11,0,...,0.014667,-0.014455,0.014667,1.099998,150.0,-0.014455,-1.099998,-0.007391,230.0,0.002464
1,3000.0,13.0,78.0,7,0,3000,13.5,80.0,11,0,...,0.115000,-0.103139,0.129114,10.199997,158.0,-0.114350,-10.199997,0.124370,238.0,-0.041457
2,3000.0,13.0,78.0,7,0,3000,13.5,80.0,11,0,...,0.163750,-0.140709,0.178481,14.099998,158.0,-0.151450,-14.099998,0.173529,238.0,-0.057843
3,6000.0,18.0,90.0,7,0,6000,18.0,90.0,13,0,...,-0.057778,0.061321,-0.057778,-5.199997,180.0,0.061321,5.199997,-0.057778,270.0,0.019259
4,6000.0,14.0,90.0,7,0,6000,14.5,86.0,11,0,...,0.119767,-0.106957,0.094318,8.300003,176.0,-0.086189,-8.300003,0.102672,262.0,-0.034224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234451,6000.0,18.0,80.0,6,0,6000,18.0,80.0,11,0,...,0.163188,-0.140293,0.163188,13.055000,160.0,-0.140293,-13.055000,0.163188,240.0,-0.054396
234452,3000.0,13.5,80.0,7,0,3000,14.5,80.0,11,0,...,0.077500,-0.071926,0.077500,6.199997,160.0,-0.071926,-6.199997,0.077500,240.0,-0.025833
234453,3000.0,13.0,75.0,8,30,3000,13.0,75.0,13,0,...,0.358667,-0.263984,0.358667,26.900002,150.0,-0.263984,-26.900002,0.358667,225.0,-0.119556
234454,6000.0,18.0,90.0,7,0,6000,18.0,90.0,13,0,...,-0.044444,0.046512,-0.044444,-4.000000,180.0,0.046512,4.000000,-0.044444,270.0,0.014815


In [4]:
all_X = pandas.concat([train_X, val_X], ignore_index=True, sort=False)
all_X = all_X.drop(columns=["B_LIGHT2","C_LIGHT2"])
test_X = test_X.drop(columns=["B_LIGHT2","C_LIGHT2"])
all_X

,A_CO2,A_TEMP,A_HUMI,A_LIGHT1,A_LIGHT2,B_CO2,B_TEMP,B_HUMI,B_LIGHT1,CO2,...,diffC_HUMI,rdiffC_HUMI,diffCA_HUMI,diffvCA_HUMI,aggCA_HUMI,rdiffCA_HUMI,rdiffvCA_HUMI,diff_HUMI,aggABC_HUMI,rdiff_HUMI
0,5000.0,18.0,75.0,7,0,5000,18.0,80.0,11,6338.000000,...,0.014667,-0.014455,0.014667,1.099998,150.0,-0.014455,-1.099998,-0.007391,230.0,0.002464
1,3000.0,13.0,78.0,7,0,3000,13.5,80.0,11,3000.000000,...,0.115000,-0.103139,0.129114,10.199997,158.0,-0.114350,-10.199997,0.124370,238.0,-0.041457
2,3000.0,13.0,78.0,7,0,3000,13.5,80.0,11,2138.000000,...,0.163750,-0.140709,0.178481,14.099998,158.0,-0.151450,-14.099998,0.173529,238.0,-0.057843
3,6000.0,18.0,90.0,7,0,6000,18.0,90.0,13,6488.200000,...,-0.057778,0.061321,-0.057778,-5.199997,180.0,0.061321,5.199997,-0.057778,270.0,0.019259
4,6000.0,14.0,90.0,7,0,6000,14.5,86.0,11,6094.000000,...,0.119767,-0.106957,0.094318,8.300003,176.0,-0.086189,-8.300003,0.102672,262.0,-0.034224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312604,3000.0,20.0,90.0,7,0,3000,21.0,90.0,11,1152.000000,...,0.160465,-0.138277,0.134091,11.800003,176.0,-0.118236,-11.800003,0.125564,266.0,-0.041855
312605,3000.0,18.0,90.0,6,0,3000,18.0,90.0,11,6522.466797,...,0.055759,-0.052814,0.055759,5.018333,180.0,-0.052814,-5.018333,0.055759,270.0,-0.018586
312606,5000.0,18.0,75.0,7,0,5000,18.0,80.0,11,6274.000000,...,0.040000,-0.038462,0.040000,3.000000,150.0,-0.038462,-3.000000,0.017391,230.0,-0.005797
312607,3000.0,13.0,80.0,6,0,3000,13.5,80.0,11,6520.000000,...,0.223312,-0.182547,0.223312,17.864998,160.0,-0.182547,-17.864998,0.223312,240.0,-0.074437


In [6]:
for i in ["A", "B", "C"]:
    print(i)
    abs_humid = []
    df_tmp = pandas.DataFrame()
    for j in range(0,len(all_X)):
        if j % 10000 == 0:
            print(j)
        Kelvin = all_X[i + "_TEMP"][j] + 273.15
        rh = all_X[i + "_HUMI"][j]
        abs_humid.append(atmos.calculate('AH', RH=rh, p=1e5, T=Kelvin))
    df_tmp = df_tmp.append(abs_humid)
    all_X[i + "_abs_humid"] = df_tmp
    
abs_humid = []
df_tmp = pandas.DataFrame()
for j in range(0,len(all_X)):
    if j % 10000 == 0:
        print(j)
    Kelvin = all_X["TEMP"][j] + 273.15
    rh = all_X["HUMI"][j]
    abs_humid.append(atmos.calculate('AH', RH=rh, p=1e5, T=Kelvin))
df_tmp = df_tmp.append(abs_humid)
all_X["abs_humid"] = df_tmp
all_X

A
0
B
0
C
0
0


,A_CO2,A_TEMP,A_HUMI,A_LIGHT1,A_LIGHT2,B_CO2,B_TEMP,B_HUMI,B_LIGHT1,CO2,...,aggCA_HUMI,rdiffCA_HUMI,rdiffvCA_HUMI,diff_HUMI,aggABC_HUMI,rdiff_HUMI,A_abs_humid,B_abs_humid,C_abs_humid,abs_humid
0,5000.0,18.0,75.0,7,0,5000,18.0,80.0,11,6338.000000,...,150.0,-0.014455,-1.099998,-0.007391,230.0,0.002464,0.011573,0.012332,0.011573,0.011670
1,3000.0,13.0,78.0,7,0,3000,13.5,80.0,11,3000.000000,...,158.0,-0.114350,-10.199997,0.124370,238.0,-0.041457,0.008870,0.009380,0.009094,0.009515
2,3000.0,13.0,78.0,7,0,3000,13.5,80.0,11,2138.000000,...,158.0,-0.151450,-14.099998,0.173529,238.0,-0.057843,0.008870,0.009380,0.009094,0.010240
3,6000.0,18.0,90.0,7,0,6000,18.0,90.0,13,6488.200000,...,180.0,0.061321,5.199997,-0.057778,270.0,0.019259,0.013845,0.013845,0.013845,0.013059
4,6000.0,14.0,90.0,7,0,6000,14.5,86.0,11,6094.000000,...,176.0,-0.086189,-8.300003,0.102672,262.0,-0.034224,0.010866,0.010714,0.010390,0.012273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312604,3000.0,20.0,90.0,7,0,3000,21.0,90.0,11,1152.000000,...,176.0,-0.118236,-11.800003,0.125564,266.0,-0.041855,NaN,NaN,NaN,NaN
312605,3000.0,18.0,90.0,6,0,3000,18.0,90.0,11,6522.466797,...,180.0,-0.052814,-5.018333,0.055759,270.0,-0.018586,NaN,NaN,NaN,NaN
312606,5000.0,18.0,75.0,7,0,5000,18.0,80.0,11,6274.000000,...,150.0,-0.038462,-3.000000,0.017391,230.0,-0.005797,NaN,NaN,NaN,NaN
312607,3000.0,13.0,80.0,6,0,3000,13.5,80.0,11,6520.000000,...,160.0,-0.182547,-17.864998,0.223312,240.0,-0.074437,NaN,NaN,NaN,NaN


In [8]:
for i in ["A", "B", "C"]:
    print(i)
    abs_humid = []
    df_tmp = pandas.DataFrame()
    for j in range(0,len(test_X)):
        if j % 10000 == 0:
            print(j)
        Kelvin = test_X[i + "_TEMP"][j] + 273.15
        rh = test_X[i + "_HUMI"][j]
        abs_humid.append(atmos.calculate('AH', RH=rh, p=1e5, T=Kelvin))
    df_tmp = df_tmp.append(abs_humid)
    test_X[i + "_abs_humid"] = df_tmp
abs_humid = []
df_tmp = pandas.DataFrame()
for j in range(0,len(test_X)):
    if j % 10000 == 0:
        print(j)
    Kelvin = test_X["TEMP"][j] + 273.15
    rh = test_X["HUMI"][j]
    abs_humid.append(atmos.calculate('AH', RH=rh, p=1e5, T=Kelvin))
df_tmp = df_tmp.append(abs_humid)
test_X["abs_humid"] = df_tmp
test_X

A
0
B
0
C
0
0


,A_CO2,A_TEMP,A_HUMI,A_LIGHT1,A_LIGHT2,B_CO2,B_TEMP,B_HUMI,B_LIGHT1,CO2,...,aggCA_HUMI,rdiffCA_HUMI,rdiffvCA_HUMI,diff_HUMI,aggABC_HUMI,rdiff_HUMI,A_abs_humid,B_abs_humid,C_abs_humid,abs_humid
0,3000,12.0,88.0,8,0,4000.0,12.0,86.0,15,3686.000000,...,176.0,0.001138,0.099998,0.006489,262.0,-0.002163,0.009388,0.009178,0.009388,0.008919
1,6000,17.0,90.0,7,0,6000.0,17.0,90.0,11,5876.000000,...,176.0,-0.090909,-8.800003,0.091729,266.0,-0.030576,0.013040,0.013040,0.012470,0.013923
2,6000,18.0,80.0,6,0,6000.0,18.0,80.0,11,1288.500000,...,160.0,-0.138287,-12.838333,0.160479,240.0,-0.053493,0.012332,0.012332,0.012332,0.013506
3,6000,18.0,90.0,7,0,6000.0,18.0,90.0,11,4968.000000,...,176.0,-0.062833,-5.900002,0.059023,266.0,-0.019674,0.013845,0.013845,0.013240,0.013925
4,6000,14.5,90.0,7,0,6000.0,15.0,86.0,11,5816.000000,...,176.0,-0.059829,-5.599998,0.071756,262.0,-0.023919,0.011205,0.011047,0.010714,0.011935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78148,4000,12.0,80.0,8,0,4000.0,13.0,75.0,16,3454.000000,...,160.0,-0.150743,-14.199997,0.202553,235.0,-0.067518,NaN,NaN,NaN,NaN
78149,5000,13.0,90.0,7,0,5000.0,13.5,90.0,13,3038.000000,...,180.0,0.062574,5.300003,-0.058889,270.0,0.019630,NaN,NaN,NaN,NaN
78150,3000,13.5,80.0,6,0,3500.0,14.0,80.0,11,2643.466553,...,160.0,-0.161572,-15.416664,0.192708,240.0,-0.064236,NaN,NaN,NaN,NaN
78151,6000,18.0,90.0,7,0,6000.0,18.0,90.0,11,4466.000000,...,176.0,-0.003398,-0.300003,-0.004135,266.0,0.001378,NaN,NaN,NaN,NaN


In [10]:
# 加入 diff_abs系列的feature
f = "abs_humid"
for t in ['A','B','C']:
    all_X['diff'+t+'_'+f] = ( all_X[f] - all_X[t+'_'+f] ) / ( all_X[t+'_'+f] )
    test_X['diff'+t+'_'+f] = ( test_X[f] - test_X[t+'_'+f] ) / ( test_X[t+'_'+f] )

In [11]:
all_X["max_CO2"] = all_X[["A_CO2", "B_CO2", "C_CO2"]].max(axis=1)
all_X["max_TEMP"] = all_X[["A_TEMP", "B_TEMP", "C_TEMP"]].max(axis=1)
all_X["max_HUMI"] = all_X[["A_HUMI", "B_HUMI", "C_HUMI"]].max(axis=1)
all_X["min_CO2"] = all_X[["A_CO2", "B_CO2", "C_CO2"]].min(axis=1)
all_X["min_TEMP"] = all_X[["A_TEMP", "B_TEMP", "C_TEMP"]].min(axis=1)
all_X["min_HUMI"] = all_X[["A_HUMI", "B_HUMI", "C_HUMI"]].min(axis=1)

In [12]:
test_X["max_CO2"] = test_X[["A_CO2", "B_CO2", "C_CO2"]].max(axis=1)
test_X["max_TEMP"] = test_X[["A_TEMP", "B_TEMP", "C_TEMP"]].max(axis=1)
test_X["max_HUMI"] = test_X[["A_HUMI", "B_HUMI", "C_HUMI"]].max(axis=1)
test_X["min_CO2"] = test_X[["A_CO2", "B_CO2", "C_CO2"]].min(axis=1)
test_X["min_TEMP"] = test_X[["A_TEMP", "B_TEMP", "C_TEMP"]].min(axis=1)
test_X["min_HUMI"] = test_X[["A_HUMI", "B_HUMI", "C_HUMI"]].min(axis=1)

In [13]:
for mm in ['max', 'min']:
    for f in ['CO2','TEMP','HUMI']:
        conditions = [
            (all_X[mm + '_' + f] == all_X['A_' + f]) & (all_X[mm + '_' + f] != all_X['B_' + f]) & (all_X[mm + '_' + f] != all_X['C_' + f]), #A
            (all_X[mm + '_' + f] == all_X['A_' + f]) & (all_X[mm + '_' + f] == all_X['B_' + f]) & (all_X[mm + '_' + f] != all_X['C_' + f]), #AB
            (all_X[mm + '_' + f] != all_X['A_' + f]) & (all_X[mm + '_' + f] == all_X['B_' + f]) & (all_X[mm + '_' + f] != all_X['C_' + f]), #B
            (all_X[mm + '_' + f] != all_X['A_' + f]) & (all_X[mm + '_' + f] == all_X['B_' + f]) & (all_X[mm + '_' + f] == all_X['C_' + f]), #BC
            (all_X[mm + '_' + f] != all_X['A_' + f]) & (all_X[mm + '_' + f] != all_X['B_' + f]) & (all_X[mm + '_' + f] == all_X['C_' + f]), #C
            (all_X[mm + '_' + f] == all_X['A_' + f]) & (all_X[mm + '_' + f] != all_X['B_' + f]) & (all_X[mm + '_' + f] == all_X['C_' + f]), #AC
            (all_X[mm + '_' + f] == all_X['A_' + f]) & (all_X[mm + '_' + f] == all_X['B_' + f]) & (all_X[mm + '_' + f] == all_X['C_' + f]) #ABC
        ]
        values = [0, 1, 2, 3, 4, 5, 6]
        all_X[mm + '_' + f + '_time'] = numpy.select(conditions, values)

In [14]:
for mm in ['max', 'min']:
    for f in ['CO2','TEMP','HUMI']:
        conditions = [
            (test_X[mm + '_' + f] == test_X['A_' + f]) & (test_X[mm + '_' + f] != test_X['B_' + f]) & (test_X[mm + '_' + f] != test_X['C_' + f]), #A
            (test_X[mm + '_' + f] == test_X['A_' + f]) & (test_X[mm + '_' + f] == test_X['B_' + f]) & (test_X[mm + '_' + f] != test_X['C_' + f]), #AB
            (test_X[mm + '_' + f] != test_X['A_' + f]) & (test_X[mm + '_' + f] == test_X['B_' + f]) & (test_X[mm + '_' + f] != test_X['C_' + f]), #B
            (test_X[mm + '_' + f] != test_X['A_' + f]) & (test_X[mm + '_' + f] == test_X['B_' + f]) & (test_X[mm + '_' + f] == test_X['C_' + f]), #BC
            (test_X[mm + '_' + f] != test_X['A_' + f]) & (test_X[mm + '_' + f] != test_X['B_' + f]) & (test_X[mm + '_' + f] == test_X['C_' + f]), #C
            (test_X[mm + '_' + f] == test_X['A_' + f]) & (test_X[mm + '_' + f] != test_X['B_' + f]) & (test_X[mm + '_' + f] == test_X['C_' + f]), #AC
            (test_X[mm + '_' + f] == test_X['A_' + f]) & (test_X[mm + '_' + f] == test_X['B_' + f]) & (test_X[mm + '_' + f] == test_X['C_' + f]) #ABC
        ]
        values = [0, 1, 2, 3, 4, 5, 6]
        test_X[mm + '_' + f + '_time'] = numpy.select(conditions, values)

In [ ]:
all_X.to_csv('./train_sensor_data_submit.csv',index=False)
test_X.to_csv('./test_sensor_data_submit.csv',index=False)